# 1.  Dealing with large state spaces

Notice one major fundamental difference (in terms of the RL components of each) between the cart-pole, lunar lander, chess, and pac-man examples and the various grid-world examples discussed previously: the size of the state-space.  For example

- With the chess example: the number of possible configurations of chess pieces on the board is enormous - i.e., the number of states or size of state space - is on the order of $10^{120}$.  That is far larger than the number of atoms in the known universe!


- With the cart-pole example: each of the four descriptors of the environment is continuous (or finely discretized in practice) - thus the number of states is - technically speaking - infinite.  


- With the Pac-man example: if we use raw pixels as input then one state is a single frame of the game, and we have as many states as there are ways to legally arrange pixels in the game.  This is an extremely large number.  


In all of these cases the (very large) size of the state space presents practical problems.  Why?  Because remember that - practically speaking - in order to learn the $Q$ properly we need to visit a representative sample of states a number of times.(theoretically speaking - we have to be able to cycle through *every possible state* infinitely often).  Again - practically speaking then - even a state space on the order of chess $10^{120}$ is far too large for us to be able to learn a representative $Q$ effectively.  

Even more pragmatically, recall from the first notebook in this series that we store the $Q$ function as a matrix of size $N \times M$, where $N$ and $M$ are the sizes of the state and action spaces respectively.  But the size of $N$ is - for the problems listed above is far too large (or infinite) to store in memory.

In sum the issue here is that in most RL problems the **size of the state space** makes directly computing and storing the $Q$ function/matrix impossible.  In this section we discuss the crucial role *function approximators* (neural networks, trees, and kernels) play in ameliorating this issue, allowing us to extend the Reinforcement paradigm to problems with infinitely large state spaces.

## 1.1  Remember: the Q function is a matrix

Remember that if we denote all the realized states as $S=\left\{ \sigma_{1},\sigma_{2},...,\sigma_{N}\right\} $, and likewise the set of all possible actions $ A=\left\{ \alpha_{1},\alpha_{2},...,\alpha_{M}\right\} $, then we represent $Q$ function on a computer as the $N \times M$ matrix

$$Q=\left[\begin{array}{cccc}
Q\left(\sigma_{1},\alpha_{1}\right) & Q\left(\sigma_{1},\alpha_{2}\right) & \cdots & Q\left(\sigma_{1},\alpha_{M}\right)\\
Q\left(\sigma_{2},\alpha_{1}\right) & \ddots &  & Q\left(\sigma_{2},\alpha_{M}\right)\\
\vdots &  & \ddots & \vdots\\
Q\left(\sigma_{N},\alpha_{1}\right) &  &  & Q\left(\sigma_{N},\alpha_{M}\right)
\end{array}\right]$$

When the number of states associated with a given problem gets big - like with e.g., chess, cart pole, and pacman - it is the *vertical* component of this matrix that gets big (the parameter $N$). The $m^{th}$ column of the $Q$ function matrix contains the (current) response of the system to a the $m^{th}$ realized action $alpha_m$ for each and every possible state, and so it is the height of each such column vector that is problematic for many problems.

## 1.1  Compactly representing the columns of the $Q$ matrix

We need a way to compactly represent *the vertical dimension* of $Q$ when we have a very large (and possibly infinite number) of states.  Instead of trying to compute and record every input/output pair - which is impossible once the input space is too large or infinite - we can try the next best thing: represent each column of $Q$ by a *parameterized function*.  If each parameterized function is chosen wisely this allows us to both

- overcome the problem of having to store a huge number of state-based computations, where instead this information is baked into the chosen function via many rounds of parameter tuning


- generalize the RL model to states we did not reach during training (when the number of states is very large/infinite this will certainly happen)


So if we choose a set of parameterized functions $h_1(s), ~h_2(s),...,~h_M(s)$ - which will replace the columns of our original $Q$ function matrix and so are functions state only -  then our $Q$ function matrix becomes a (row) vector of the $M$ functions as 


$$Q=\left[\begin{array}{cccc}
h_{1}\left(s\right)\,\,\,\,\,\,\,\,\, & h_{2}\left(s\right) & \,\,\,\,\,\,\,\,\cdots & h_{M}\left(s\right)\end{array}\right]$$

These parameterized functions all typically take the same form, whether it be linear, polynomial, neural network, tree-based, etc.,

How do we augment our Q-Learning scheme to ensure these parameterized functions accurately represent the data we generate via simulation?  

## 1.2  The Q-learning update step

To ensure that each of these functions accurately represents the column of relevant data we will insert into the Q-Learning scheme a new step.  After choosing an action, recieving a reward, moving to a new state, and computing an updated $Q$ value for this state/action pair we will regress this value to the corresponding function.  

Remember that with the original Q-Learning we update a single entry of the matrix at each step as

$Q\left(s_{k-1},\,a_{k}\right)=r_{k}+\gamma\cdot\underset{i=1...M}{\text{maximum}}\,\,Q\left(s_{k},\,\alpha_{i}\right)$


Now, however, we have a parameterized function taking the place of each column of $Q$, and because of this we need to split this step *into two pieces* when adapting it to our current situation.  

In the first step we generate the update quantity:  

$q_{k}=r_{k}+\gamma\cdot\underset{i=1...M}{\text{maximum}}\,\,h_{i}(s_k)$


This is precisely what we did previously - the only thing that has changed is that now our $Q$ matrix is represented by a list of parameterized functions - one for each column - and so the *maximum* in the above update step is over our functions (and not the literal columns of a matrix).  Because we have no $Q$ matrix of values we call the value on the left hand side $q_{k}$.

But we also need to update our representation of $Q$ with this data point.  In the original case, where we had no function approximators, we simply replaced the entry from the appropriate column of the $Q$ matrix with this value.  The analog here is that we now need to *re-tune* the parameters of the parameterized function taking the place of the appropriate column of $Q$.  

The best way to do this is to refit the appropriate model to this data point via solving a regularized Least Squares problem.  Letting $j$ denote the index of the maximum acheived on the right hand side of the update above, an unregularized Least Squares would mean finding the parameters $\boldsymbol{\theta}_{j}$ that minimize the quantity $(h_{j}(s_k)-q_{k})^{2}$.  However minimizing this alone would overfit our parameters to the single point $q_k$, and we would lose all of the information contained in the previously tuned parameters.  In order to avoid this we can add a simple regularizer that ensures that the new parameters aren't too different from the previous ones: denoting the previous set of parameters $\boldsymbol{\theta}_{j}^{\text{ prev}}$ we add an appropriate regularizer to the Least Squares

$\underset{{j}}{\text{minimize}}\,\,\left(h_{j}\left(s_{k}\right)-q_{k}\right)^{2}+\beta\left\Vert \boldsymbol{\theta}_{j}-\boldsymbol{\theta}_{j}^{\text{ prev}}\right\Vert _{2}^{2}$


Here $\beta$ is a parameter we can tune to ensure that the new parameters are either very close to the previously calculated parameters (by setting $\beta$ to a large positive number), or lighten this restriction by setting $\beta$ low.  Typically this is either set to be a small constant, or increased gradually at each episode of simulation.

Regardless of the $\beta$ setting, and the form of $h$, this regularized Least Squares problem has a closed form solution.  Setting the gradient in $\boldsymbol{\theta}_{j}$ above to zero and solving gives the solution

$\boldsymbol{\theta}_{j}=\boldsymbol{\theta}_{j}^{\text{ prev}}-\frac{1}{\beta}(h_{j}\left(s_{k}\right)-q_{k})\nabla h_{j}\left(s_{k}\right)$

Due to the form of this update, this is often interpreted as a stochastic gradient descent step.

## 1.3 An algorithm for Q learning with function approximators

All together a basic algorithm for Q learning with function approximators looks like

------
Initialize parameter vectors $\boldsymbol{w}_{1},...,\boldsymbol{w}_{M}$ choose a value $p\in[0,1]$, for $\gamma \in [0,1]$, and a step size rule

**for** e = 1...E (the maximum number of episodes)
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Select a random initial state $s_0$ and set $k=1$
    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **while** end state not reached AND maximum iteration count not met

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**if** $r < p$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Choose $a_k$ randomly
  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**else** 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Choose $a_k$ according to the optimal policy


  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;i.e., where $k=\underset{i=1...M}{\text{argmax}}\,\,h_{i}(s_k)$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Recieve** the next state $s_k$ and corresponding immediate reward $r_k$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;**Update** appropriate function approximator

    
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$q_{k}=r_{k}+\gamma\cdot\underset{i=1...M}{\text{maximum}}\,\,h_{i}(s_k)$


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\boldsymbol{w}_{j}=\boldsymbol{w}_{j}^{\text{ prev}}-\frac{1}{\beta}(h_{j}\left(s_{k}\right)-q_{k})\nabla h_{j}\left(s_{k}\right)$


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$k \longleftarrow k+1$

-----

We use this procedure to train an agent for the cart-pole problem in the Python cell below. When you run this cell the first few episodes of training will be animated - much as we did with the grid world example in Section 2.

## Example: cart pole

In [3]:
from my_cartpole import my_cartpole
cartpole_test = my_cartpole()
cartpole_test.qlearn(gamma=0.8,num_episodes = 100)

[2017-04-06 12:08:21,059] Making new env: CartPole-v0


q-learning process complete, best number of average steps in test runs = 199.0


You can expose the Q learning function here by activating the Python cell below - it maches the pseudo-code given above.

In [3]:
test.qlearn??

## 1.4  What is the best action per state given a learned Q function?

Given the parameterized functions representing $Q$ are learned properly, how should the agent move at each state?  Precisely as we did when with problems like grid world - where we chose the action maximizing the corresponding $Q$ value.  The only difference here is that each column of $Q$ is a parameterized function.

So at a given state $s_{k-1}$ we take the action $a_k = \alpha_j$ where the argument $j$ is given as 

$j=\underset{i=1...M}{\text{argmax}}\,\,h_{i}(s_{k-1})$

This is precisely how the learned agent controls the cart-pole, as you can see by activating the Python cell below.  

## Example: cartpole

In [4]:
# run a few test runs of cart-pole using the trained function approximators
cartpole_test.animate_test_run()
cartpole_test.animate_test_run()

lasted 199 timesteps
lasted 199 timesteps


To expose the controller activate the Python cell below.

In [4]:
test.animate_test_run??